# Visual Place Recognition: A Tutorial
This notebook shows a practical code example in Python that illustrates to prospective practitioners and researchers how VPR is implemented and evaluated. The example implements a basic VPR pipeline with the key steps and components that are part of most VPR pipelines.

For details or if you use our work for your academic research, please refer to the following paper:
```bibtex
@article{SchubertVisual,
    title={Visual Place Recognition: A Tutorial},
    author={Stefan Schubert and Peer Neubert and Sourav Garg and Michael Milford and Tobias Fischer},
    journal={arXiv 2303.03281},
    year={2023},
}
```

## Imports
Import the required libraries and functions.

In [1]:
import argparse
import configparser
import os

from evaluation.metrics import createPR, recallAt100precision, recallAtK
from evaluation import show_correct_and_wrong_matches
from matching import matching
from datasets.load_dataset import GardensPointDataset, StLuciaDataset, SFUDataset
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from IPython.display import display

from matplotlib import pyplot as plt

## Load dataset
In this example, the input to a VPR algorithm are two image sets: the database DB and query Q (i.e., multi-set VPR). For later evaluation, we also load ground-truth information about correspondences. This ground truth only serves for evaluation and will neither be available nor required when deploying the algorithm.

For demonstration, the relatively small *GardensPoint Walking* dataset with 200 images per image set, a subset of the *StLucia* dataset with 200 images can be loaded, or the 385-images dataset SFU Mountain.

In [2]:
# Widget for selecting a dataset
def select_dataset(Dataset=['GardensPoint', 'StLucia', 'SFU']):
    return Dataset
ds = interactive(select_dataset)
display(ds)

interactive(children=(Dropdown(description='Dataset', options=('GardensPoint', 'StLucia', 'SFU'), value='Garde…

In [3]:
# load dataset with ground truth
dataset_name = ds.result
if dataset_name == 'GardensPoint':
    dataset = GardensPointDataset()
elif dataset_name == 'StLucia':
    dataset = StLuciaDataset()
elif dataset_name == 'SFU':
    dataset = SFUDataset()
imgs_db, imgs_q, GThard, GTsoft = dataset.load()

===== Load dataset GardensPoint day_right--night_right


## Descriptor computation
The main source of information about image correspondences are image descriptors. Local descriptors like DELF provide information for multiple regions of interest but are computationally expensive to compare. Holistic image descriptors reduce the computational complexity, but often have slightly lower performance. Feature aggregation methods such as HDC can be used to combine the local descriptors of an image in a single holistic descriptor vector.

In the following, different holistic or local image descriptor can be selected. HDC-DELF, AlexNet-conv3 and NetVLAD are deep-learning based holistic descriptors. PatchNetVLAD is used as local descriptors in the following, so that it requires the highest runtime for creation and comparison.

In [4]:
# Widget for selecting a descriptor
def select_descriptor(Descriptor=['HDC-DELF', 'AlexNet-conv3', 'NetVLAD', 'PatchNetVLAD']):
    return Descriptor
w = interactive(select_descriptor)
display(w)

interactive(children=(Dropdown(description='Descriptor', options=('HDC-DELF', 'AlexNet-conv3', 'NetVLAD', 'Pat…

In [5]:
# select descriptor
descriptor = w.result
print(f'===== Compute {descriptor} descriptors')
feature_extractor = None
if descriptor == 'HDC-DELF':
    from feature_extraction.feature_extractor_local import HDCDELF
    feature_extractor = HDCDELF()
elif descriptor == 'AlexNet-conv3':
    from feature_extraction.feature_extractor_holistic import AlexNetConv3Extractor
    feature_extractor = AlexNetConv3Extractor()
elif descriptor == 'NetVLAD' or descriptor == 'PatchNetVLAD':
    from feature_extraction.feature_extractor_patchnetvlad import PatchNetVLADFeatureExtractor
    from patchnetvlad.tools import PATCHNETVLAD_ROOT_DIR
    if w.result == 'NetVLAD':
        configfile = os.path.join(PATCHNETVLAD_ROOT_DIR, 'configs/netvlad_extract.ini')
    else:
        configfile = os.path.join(PATCHNETVLAD_ROOT_DIR, 'configs/speed.ini')
    assert os.path.isfile(configfile)
    config = configparser.ConfigParser()
    config.read(configfile)
    feature_extractor = PatchNetVLADFeatureExtractor(config)

# compute holistic descriptors
if descriptor != 'PatchNetVLAD':
    print('===== Compute reference set descriptors')
    db_D_holistic = feature_extractor.compute_features(imgs_db)
    print('===== Compute query set descriptors')
    q_D_holistic = feature_extractor.compute_features(imgs_q)
else:
    print('=== WARNING: The PatchNetVLAD code in this repository is not optimised and will be slow and memory consuming.')
    print('===== Compute reference set descriptors')
    db_D_holistic, db_D_patches = feature_extractor.compute_features(imgs_db)
    print('===== Compute query set descriptors')
    q_D_holistic, q_D_patches = feature_extractor.compute_features(imgs_q)

===== Compute HDC-DELF descriptors


ImportError: cannot import name 'HDCDELF' from 'feature_extraction.feature_extractor_local' (/home/stefans/workspace/publications/2022/vpr_tutorial/VPR_Tutorial/feature_extraction/feature_extractor_local.py)

## Descriptor comparison and similarity matrix S
To compare database and query descriptors, we either use the cosine similarity for holistic descriptors (e.g. computed by the inner product of the normalized descriptor vectors), or a more complex algorithmic approach for local descriptors. Although we might not want to compute the full similarity matrix S of all possible pairs in a large scale practical application, it can be useful for visual inspection purposes.

In the following, the dense similarity matrix S between all descriptor pair is computed.

In [ ]:
# compare all descriptors and compute similarity matrix S
print('===== Compute cosine similarities S')
if descriptor != 'PatchNetVLAD':
    # normalize descriptors and compute S-matrix
    db_D_holistic = db_D_holistic / np.linalg.norm(db_D_holistic , axis=1, keepdims=True)
    q_D_holistic = q_D_holistic / np.linalg.norm(q_D_holistic , axis=1, keepdims=True)
    S = np.matmul(db_D_holistic , q_D_holistic.transpose())
else:
    S = feature_extractor.local_matcher_from_numpy_single_scale(q_D_patches, db_D_patches)

# show similarity matrix S
fig = plt.imshow(S)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)
fig.axes.set_title('Similarity matrix S')

The image shows the similarity matrix $S{\in}\mathbb{R}^{|DB|\times|Q|}$. In the GardensPoint Walking dataset, images with the same ID were recorded at same places, i.e., the i-th database image matches the i-th query image. Therefore, we can observe high similarities along the main diagonal of S.

## Image matching
The output of a VPR pipeline is typically a set of discrete matchings, i.e. pairs of query and database images. To obtain matchings for a query image from the similarity matrix, we can either find the single best matching database image (M1) or try to find all images in the database that show the same place as the query image (M2).

In [ ]:
# best matching per query in S for single-best-match VPR
M1 = matching.best_match_per_query(S)

# find matches with S>=thresh using an auto-tuned threshold for multi-match VPR
M2 = matching.thresholding(S, 'auto')
TP = np.argwhere(M2 & GThard)  # true positives
FP = np.argwhere(M2 & ~GTsoft)  # false positives

# show M's
fig = plt.figure()
ax1 = fig.add_subplot(121)
ax1.imshow(M1)
ax1.axis('off')
ax1.set_title('Best match per query')
ax2 = fig.add_subplot(122)
ax2.imshow(M2)
ax2.axis('off')
ax2.set_title('Thresholding S>=thresh')

Both matrices $M{\in}\mathbb{R}^{|DB|\times|Q|}$ show the matched images between the database and the query set. Left, only the best match per query was selected, leading to a thin line. Right, all image pairs with a similarity above a threshold were selected, s.t. we can also see multiple or no matches per query image.

In [ ]:
# show correct and wrong image matches
show_correct_and_wrong_matches.show(imgs_db, imgs_q, TP, FP)  # show random matches
plt.title('Examples for correct and wrong matches from S>=thresh')

The green frame shows a correctly matched image pair, the red frame a wrongly matched image pair.

## Evaluation
To evaluate the quality of a similarity matrix S, we can apply a series of decreasing thresholds $\theta$ to match more and more image pairs. Combined with ground-truth information, each threshold leads to a different set of true positives, false positives, true negatives and false negatives, which then provides one point on the precision-recall curve.

In the following, the precision-recall curve and the area under the precision-recall curve is computed and visualized for *multi-match VPR*, i.e. all matches between each query image and the database have to be identified.

In [ ]:
# precision-recall curve
P, R = createPR(S, GThard, GTsoft, matching='multi', n_thresh=100)
plt.figure()
plt.plot(R, P)
plt.xlim(0, 1), plt.ylim(0, 1.01)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Result on GardensPoint day_right--night_right')
plt.grid('on')
plt.draw()

The graph shows the precision-recall curve. A curve closer to the upper right corner would represent better performance. Precision=1 means that no false positives (FP) were extracted. A recall=1 means that all same places were found.

In [ ]:
# area under precision-recall curve (AUPRC)
AUPRC = np.trapz(P, R)
print(f'\n===== AUPRC: {AUPRC:.3f}')

The AUPRC performance ranges between 0 and 1 (higher is better).

In [ ]:
# maximum recall at 100% precision
maxR = recallAt100precision(S, GThard, GTsoft, matching='multi', n_thresh=100)
print(f'\n===== R@100P (maximum recall at 100% precision): {maxR:.3f}')

The maximum recall at 100% precision ranges between 0 and 1 (higher is better).

In [ ]:
# recall@K
Rat1 = recallAtK(S, GThard, GTsoft, K=1)
Rat5 = recallAtK(S, GThard, GTsoft, K=5)
Rat10 = recallAtK(S, GThard, GTsoft, K=10)
print(f'\n===== recall@K (R@K): R@1: {Rat1:.3f}, R@5: {Rat5:.3f}, R@10: {Rat10:.3f}')

The recall@K ranges between 0 and 1 (higher is better). The recall@K measures the rate of query images with at least one actually matching database image. Accordingly, the metric gets better with increasing K.